# master simulations

All simulations needed for Phasics imagelets:

1. taking a bunch of images 
2. compute edges and
3. then doing statistics on that
4. to see if we can classify them.

## types of simulations 

We do different levels of simulations:

- the basic one with defaults parameters
- testing diferent control parameters at the level of the co-occurence descriptor
- testing different parameters for parameter extraction


In [1]:
%load_ext autoreload
%autoreload 2

# running everything

In [2]:
experiment_folder = 'BICV-Sparse'

cluster = True
cluster = False

if cluster:
    try:
        from INT_cluster import Frioul
        k = Frioul(experiment_folder, N_jobs = 64)
    except Exception:
        cluster = False
else:
    def run_cmd(cmd, doit=True):
        import subprocess
        print ('⚡︎ Running ⚡︎ ', cmd)
        if doit:
            stdout = subprocess.check_output([cmd], shell=True)
            return stdout.decode()#.splitlines()

do_update = True
do_update = False

do_cleanup = False
do_cleanup = True

do_run = False
do_run = True

experiments = ['experiment_' + name +  '.py' for name in ['sparselets', 'efficiency', 'firstorder']]#, 'secondorder']]
#experiments = ['experiment_' + name +  '.py' for name in ['sparselets']]

In [3]:
if cluster and do_cleanup:
    for cmd in [
        #"rm -fr results data_cache ",
        #"find . -name *sparselets* -exec rm -fr {} \\;",
        "find . -name *lock* -exec rm -fr {} \\;",
        #"touch frioul; rm frioul* ",
        ]:
        print(k.run_on_cluster(cmd))


# preparing
if do_run:
    # RUNNING
    if cluster: k.push_to_cluster()
    for experiment in experiments:
        if cluster:
            fullcmd = 'ipython {experiment}'.format(experiment=experiment)
            for cmd in [
                "frioul_batch  -M {N_jobs} '{cmd}' ".format(N_jobs=k.N_jobs, cmd=fullcmd), 
                "frioul_list_jobs -v |grep job_array_id |uniq -c",
                        ]:
                print(k.run_on_cluster(cmd))
        else:
            fullcmd = 'ipython3 {experiment}'.format(experiment=experiment)
            run_cmd (fullcmd)

⚡︎ Running ⚡︎  ipython3 experiment_sparselets.py
⚡︎ Running ⚡︎  ipython3 experiment_efficiency.py
⚡︎ Running ⚡︎  ipython3 experiment_firstorder.py


In [4]:
# GETTING the data
import time, os
if cluster:
    while True:    
        print(k.pull_from_cluster())
        print(k.run_on_cluster("tail -n 10 {}".format(os.path.join(k.PATH, 'debug.log'))))
        print(k.run_on_cluster("frioul_list_jobs -v |grep job_array_id |uniq -c"))
        locks = k.run_cmd ("find ../test -name *lock -exec ls -l {} \;")
        print(locks)
        if len(locks) == 0: break
        time.sleep(100)